<a href="https://colab.research.google.com/github/ameympatil/MIT-ML/blob/main/A2_HyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras import layers
from keras_tuner.tuners import RandomSearch

In [ ]:
df = pd.read_csv('/content/data.csv') 
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,Seattle,WA 98119,USA
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,Bellevue,WA 98008,USA
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,Redmond,WA 98052,USA


In [ ]:
df.shape

(4600, 17)

In [ ]:
df.isnull().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   price          4600 non-null   float64
 1   bedrooms       4600 non-null   int64  
 2   bathrooms      4600 non-null   float64
 3   sqft_living    4600 non-null   int64  
 4   sqft_lot       4600 non-null   int64  
 5   floors         4600 non-null   float64
 6   waterfront     4600 non-null   int64  
 7   view           4600 non-null   int64  
 8   condition      4600 non-null   int64  
 9   sqft_above     4600 non-null   int64  
 10  sqft_basement  4600 non-null   int64  
 11  yr_built       4600 non-null   int64  
 12  yr_renovated   4600 non-null   int64  
 13  street         4600 non-null   object 
 14  city           4600 non-null   object 
 15  statezip       4600 non-null   object 
 16  country        4600 non-null   object 
dtypes: float64(3), int64(10), object(4)
memory usage: 61

In [ ]:
len(df.city.unique())

44

In [ ]:
len(df.street.unique())

4525

In [ ]:
df.city.value_counts().to_dict()

{'Algona': 5,
 'Auburn': 176,
 'Beaux Arts Village': 1,
 'Bellevue': 286,
 'Black Diamond': 9,
 'Bothell': 33,
 'Burien': 74,
 'Carnation': 22,
 'Clyde Hill': 11,
 'Covington': 43,
 'Des Moines': 58,
 'Duvall': 42,
 'Enumclaw': 28,
 'Fall City': 11,
 'Federal Way': 148,
 'Inglewood-Finn Hill': 1,
 'Issaquah': 187,
 'Kenmore': 66,
 'Kent': 185,
 'Kirkland': 187,
 'Lake Forest Park': 36,
 'Maple Valley': 96,
 'Medina': 11,
 'Mercer Island': 86,
 'Milton': 2,
 'Newcastle': 33,
 'Normandy Park': 18,
 'North Bend': 50,
 'Pacific': 6,
 'Preston': 2,
 'Ravensdale': 7,
 'Redmond': 235,
 'Renton': 293,
 'Sammamish': 175,
 'SeaTac': 29,
 'Seattle': 1573,
 'Shoreline': 123,
 'Skykomish': 3,
 'Snoqualmie': 71,
 'Snoqualmie Pass': 1,
 'Tukwila': 29,
 'Vashon': 29,
 'Woodinville': 115,
 'Yarrow Point': 4}

In [ ]:
freq_map = df.city.value_counts().to_dict()
df.city = df.city.map(freq_map)
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,123,WA 98133,USA
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,1573,WA 98119,USA
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,185,WA 98042,USA
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,286,WA 98008,USA
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,235,WA 98052,USA


In [ ]:
freq_map = df.statezip.value_counts().to_dict()
df.statezip = df.statezip.map(freq_map)
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,123,93,USA
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,1573,49,USA
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,185,100,USA
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,286,50,USA
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,235,135,USA


In [ ]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,18810 Densmore Ave N,123,93,USA
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,709 W Blaine St,1573,49,USA
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,26206-26214 143rd Ave SE,185,100,USA
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,857 170th Pl NE,286,50,USA
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,9105 170th Ave NE,235,135,USA


In [ ]:
df = df.drop(columns=['street','country'],axis='columns')
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city,statezip
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,123,93
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,1573,49
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,185,100
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,286,50
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,235,135


In [ ]:
X = df.drop(columns=['price'],axis='columns')
y = df['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [ ]:
x_train= X_train.values.reshape(-1, 1)
x_test = X_test.values.reshape(-1, 1)
y_train= y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [ ]:
scalar = MinMaxScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)
y_train = scalar.fit_transform(y_train)
y_test = scalar.transform(y_test)

In [ ]:
X_train

array([[0.66666667, 0.22222222, 0.16132368, ..., 0.        , 0.04643766,
        0.36054422],
       [0.22222222, 0.33333333, 0.08893485, ..., 0.94935452, 1.        ,
        0.53741497],
       [0.33333333, 0.22222222, 0.11375388, ..., 0.97914598, 0.11704835,
        0.67346939],
       ...,
       [0.33333333, 0.14814815, 0.08066184, ..., 0.97219464, 0.07760814,
        0.59183673],
       [0.22222222, 0.14814815, 0.05584281, ..., 0.99553128, 1.        ,
        0.46258503],
       [0.22222222, 0.14814815, 0.11582213, ..., 0.        , 1.        ,
        0.89115646]])

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=1,
                                            max_value=1000,
                                            step=100),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='final',
    project_name='Price Prediction')

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 1000, 'step': 100, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 1000, 'step': 100, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
tuner.search(X_train,y_train,epochs=10,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 16s]
val_mean_absolute_error: 0.025601410617431004

Best val_mean_absolute_error So Far: 0.025601410617431004
Total elapsed time: 00h 01m 31s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

Results summary
Results in final/Price Prediction
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 2
units_0: 201
units_1: 901
learning_rate: 0.01
units_2: 801
units_3: 801
units_4: 901
units_5: 101
units_6: 801
units_7: 501
Score: 0.025601410617431004
Trial summary
Hyperparameters:
num_layers: 5
units_0: 501
units_1: 201
learning_rate: 0.0001
units_2: 701
units_3: 1
units_4: 701
Score: 0.02866370106736819
Trial summary
Hyperparameters:
num_layers: 8
units_0: 901
units_1: 201
learning_rate: 0.001
units_2: 901
units_3: 801
units_4: 401
units_5: 1
units_6: 1
units_7: 1
Score: 0.0359111155072848
Trial summary
Hyperparameters:
num_layers: 5
units_0: 501
units_1: 201
learning_rate: 0.01
units_2: 101
units_3: 801
units_4: 1
Score: 0.03591246654589971
Trial summary
Hyperparameters:
num_layers: 5
units_0: 701
units_1: 401
learning_rate: 0.0001
units_2: 1
units_3: 1
units_4: 1
Score: 0.035965784142414726
